In [7]:
import torch
import csv
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import albumentations as A

In [31]:
cars_annotations = "/home.stud/kuntluka/dataset/cars/data/train_solution_bounding_boxes.csv"
df = pd.read_csv(cars_annotations, index_col='image')
df.head()
print(len(df.index.unique().values))
# print(len(df.index.values))
imgs = df.index.unique().values
df.loc[imgs[2]]
df.loc['vid_4_9800.jpg']
# df.image
# df[0.:]

# df[0]
# ann_file = csv.reader(cars_annotations)
# ann_file[0]
# df.tail(10)

355


,xmin,ymin,xmax,ymax
image,,,,
vid_4_9800.jpg,18.709395,192.414994,97.97815,231.301004
vid_4_9800.jpg,630.703569,180.109294,676.00000,214.073025


In [105]:
from PIL import Image
from albumentations.pytorch import ToTensorV2
from pathlib import Path
import numpy as np
class CarsDataset(Dataset):
    def __init__(self, img_root, ann_file):
        super().__init__()
        self.root = Path(img_root)
        self.df = pd.read_csv(ann_file, index_col='image')
        # self.ids = list(df.index.unique().values)
        self.ids = [_.name for _ in self.root.iterdir()]
        self.ids_with_box = self.df.index.unique().values
        self.transforms = A.Compose([
            A.Resize(width=480, height=480),
            A.Normalize(mean=(0,0,0), std=(1,1,1)),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'], min_visibility=0.4, min_area=5))
        self.num_cat = 1
        self.imgs = [Image.open(self.root / p) for p in self.ids]

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.ids[index] in self.ids_with_box:
            target = self.df.loc[self.ids[index]].values
        else:
            target = np.array([])

        if target.ndim == 1:
            target = np.expand_dims(target, axis=0)
        n_boxes = np.size(target, 0)
        if np.size(target, 1) == 0:
            n_boxes = 0
            target = []

        labels = [0 for _ in range(n_boxes)]
        transformed = self.transforms(image=np.array(img), bboxes=target, class_labels=labels)
        target = {}
        img = transformed['image']
        target["boxes"] = torch.tensor(transformed["bboxes"])
        target["labels"] = torch.tensor(transformed["class_labels"])

        return img, target

    
    def __len__(self) -> int:
        return len(self.ids)

In [113]:
def get_dataset_mean_std(dataloader : DataLoader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in dataloader:
        data = torch.stack(data)
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches
    std = (channels_squared_sum/num_batches - mean**2)**0.5
    return mean, std

In [115]:
cars_annotations = "/home.stud/kuntluka/dataset/cars/data/train_solution_bounding_boxes.csv"
training_root = "/home.stud/kuntluka/dataset/cars/data/training_images"
dataset = CarsDataset(training_root, cars_annotations)
# dataset[1]

def collate(batch):
    return tuple(zip(*batch))
dl = DataLoader(dataset, 8, collate_fn=collate)
# next(iter(dl))
mean, std = get_dataset_mean_std(dl)
print(f"mean : {mean} , std : {std}")
# all_y = {}
# for i in range(300):
#     x,y = dataset[i]
#     all_y[i] = y['labels']

        
    # if len(y['labels']) == 0:
    #     print(y)
    # if len(y['labels']) == 2:
    #     print(y)
    # print(dataset[i])

mean : tensor([0.2432, 0.3255, 0.3522]) , std : tensor([0.2186, 0.2592, 0.3083])


In [83]:
print(all_y)

{0: tensor([0, 0]), 1: tensor([]), 2: tensor([0, 0, 0, 0]), 3: tensor([]), 4: tensor([]), 5: tensor([0, 0]), 6: tensor([0]), 7: tensor([]), 8: tensor([0, 0, 0]), 9: tensor([]), 10: tensor([0]), 11: tensor([0]), 12: tensor([]), 13: tensor([]), 14: tensor([]), 15: tensor([0]), 16: tensor([]), 17: tensor([]), 18: tensor([]), 19: tensor([]), 20: tensor([]), 21: tensor([]), 22: tensor([]), 23: tensor([]), 24: tensor([]), 25: tensor([]), 26: tensor([]), 27: tensor([0, 0]), 28: tensor([]), 29: tensor([]), 30: tensor([0, 0, 0]), 31: tensor([]), 32: tensor([]), 33: tensor([]), 34: tensor([]), 35: tensor([]), 36: tensor([]), 37: tensor([]), 38: tensor([]), 39: tensor([0]), 40: tensor([0]), 41: tensor([0, 0]), 42: tensor([0, 0]), 43: tensor([0]), 44: tensor([]), 45: tensor([0, 0, 0, 0, 0]), 46: tensor([0]), 47: tensor([]), 48: tensor([0]), 49: tensor([]), 50: tensor([]), 51: tensor([]), 52: tensor([0]), 53: tensor([0]), 54: tensor([0, 0, 0]), 55: tensor([0]), 56: tensor([0]), 57: tensor([0]), 58: